## CS4287 - Neural Network
### Assignment 1 - 4th Year Semester 1 2024

Adam Collins: 21332967

Italo da Silva: 21326312

The Code executes to the end without an error. 

In [ ]:
# Imports

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score

In [ ]:
# 1. The Data Set 
#   (a). Visualisation of key attributes.

# Loading the csv file and viewing first rows using pandas library
housing_data = pd.read_csv('Melbourne_housing_FULL.csv', low_memory=False) 

# Filtering the Dataset
# Dropping every cell that contains a NaN value (there was alot)
columns = ["Suburb","Rooms","Type","Price","Method","SellerG","Date","Distance","Bedroom2","Bathroom","Car","Landsize","BuildingArea","YearBuilt","CouncilArea","Lattitude","Longtitude","Regionname","Propertycount"]
housing_data = housing_data.dropna(subset=columns)

# Dropping all the columns that don't contain numbers (there was alot too )
housing_data.drop(["Suburb", "Address", "Type", "Method", "SellerG", "CouncilArea", "Regionname", "Date"], inplace=True, axis="columns")
housing_data.head()

In [ ]:
# Draw graph of correlation between Price and Building Area

## Converting the data in the columns to numeric 
housing_data[['Price', 'BuildingArea']] = housing_data[['Price', 'BuildingArea']].apply(pd.to_numeric)

## Reshaping the data to a 2D array in order to plot it in the Linear Regression 
price = housing_data['Price'].values.reshape(-1,1)
building_area = housing_data['BuildingArea'].values.reshape(-1,1)

## Creating a Linear Regression model to predict the data for Building Area based on the Price
reg = LinearRegression().fit(price, building_area)

## Functions to plot the graph
plt.plot(housing_data[['Price']], housing_data[['BuildingArea']], 'o')
plt.plot(housing_data[['Price']], reg.predict(price), 'r')

## Adding labels to the graph
plt.xlabel('Price')
plt.ylabel('Building Area')
plt.title('Price vs Building Area')
plt.show()

In [ ]:
# From sample Assignment 1 provided by the lecturer.

# Dataframe of key attributes
housing_data.corr()

# Correlation matrix
corr_matrix = housing_data.corr()

# Generate Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='jet', cbar=True, fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# # 1. The Data Set 
#   (b). Pre-Processing - Normalisation

# Normalizing the data
scaler = MinMaxScaler()
housing_data[['Rooms', 'Price', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Propertycount']] = scaler.fit_transform(housing_data[['Rooms', 'Price', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Propertycount']])

# Only keeping the BuildingArea, Bathroom, Bedroom2 and YearBuilt columns as they are closest correlations to the Price
housing_data.drop(["Rooms", "Distance", "Postcode", "Car", "Landsize", "Lattitude", "Longtitude", "Propertycount"], inplace=True, axis="columns")

housing_data.head()

In [ ]:
# Select the features and target variable
features = ['BuildingArea', 'Bedroom2', 'Bathroom', 'YearBuilt']
target = 'Price'

x = housing_data[features]
y = housing_data[target]

# In order to validate and evaluate the model, we are splitting the dataset into two parts, training and testing data
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

print("x_train shape: ", x_train.shape[0])
print("x_test shape: ", x_test.shape[0])

In [ ]:
# Define the model
model = tf.keras.models.Sequential()

# Add layers to the model
model.add(tf.keras.layers.Dense(128, input_shape=(len(features),), activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1))  # Output layer for regression

# Compile the model with mean_squared_error loss function
model.compile(optimizer='adam', loss='MSE', metrics=['mae'])

# Print the model summary
model.summary()

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=100, verbose=1, validation_data=(x_test, y_test))